# Headphones V2 - Evaluation

In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from scipy.integrate import simps
import matplotlib.pyplot as plt
import os, glob, re
from pathlib import Path
from datetime import timedelta, datetime
import neurokit2 as nk
from asrpy import ASR
from scipy.signal import spectrogram
import flows

# settings to display all columns
pd.set_option("display.max_columns", None)
# Set the number of rows to display
# pd.set_option('display.max_rows', 50)  # Show up to 100 rows

# General file locations
main_folder = "../Data/"

# Session Overview

In [2]:
# Loading the manually created mapping file
sessions = pd.read_csv(main_folder+"processed/session_overview.csv")
sessions.head(5)

,Session,ID,Config,Folder,Run,Rec_folder
0,2025-07-04-01,bk7zlu,Ear Only,GelledEarOnly/2025-07-04/bk7zlu/,rec1,GelledEarOnly/2025-07-04/bk7zlu/rec1/
1,2025-07-04-01,bk7zlu,Ear Only,GelledEarOnly/2025-07-04/bk7zlu/,rec2,GelledEarOnly/2025-07-04/bk7zlu/rec2/
2,2025-07-04-01,bk7zlu,Ear Only,GelledEarOnly/2025-07-04/bk7zlu/,rec3,GelledEarOnly/2025-07-04/bk7zlu/rec3/
3,2025-07-04-01,aq9b9k,Top+Ears,DryWithHeadband/2025-07-04/aq9b9k/,rec1,DryWithHeadband/2025-07-04/aq9b9k/rec1/
4,2025-07-04-01,aq9b9k,Top+Ears,DryWithHeadband/2025-07-04/aq9b9k/,rec2,DryWithHeadband/2025-07-04/aq9b9k/rec2/


# oTree Logs & Functions

In [3]:
# Load otree log for last sessions
otree_df = pd.read_csv(main_folder + "processed/otree_logs.csv")
otree_df.head(3)

,participant.code,participant._index_in_pages,participant._max_page_index,participant.time_started_utc,participant.token,session.config.name,setup.1.player.headset_color,setup.1.player.token,introduction.1.player.handedness,introduction.1.player.english,introduction.1.player.multilingual,introduction.1.player.age,introduction.1.player.gender,introduction.1.player.glasses,introduction.1.player.eye_sight,introduction.1.player.color_sight,introduction.1.player.hearing,introduction.1.player.occupation,introduction.1.player.education_level,introduction.1.player.headsize,introduction.1.player.weight,introduction.1.player.height,introduction.1.player.hair_style_top,introduction.1.player.hair_style_ears,introduction.1.player.hair_type,introduction.1.player.hair_density,introduction.1.player.beard_style,introduction.1.player.hair_products,introduction.1.player.skin_oily_dry_1,introduction.1.player.skin_oily_dry_2,introduction.1.player.skin_resistant_sensitive_1,introduction.1.player.skin_resistant_sensitive_2,introduction.1.player.skin_products,introduction.1.player.time_since_meal,introduction.1.player.time_since_hydration,introduction.1.player.time_since_caffeine,introduction.1.player.time_since_nicotine,introduction.1.player.time_since_alcohol,introduction.1.player.flow_trait_single,introduction.1.player.flow_trait_sfds1,introduction.1.player.flow_trait_sfds2,introduction.1.player.flow_trait_sfds3,introduction.1.player.flow_trait_sfds4,introduction.1.player.flow_trait_sfds5,introduction.1.player.flow_trait_sfds6,introduction.1.player.flow_trait_sfds7,introduction.1.player.flow_trait_sfds8,introduction.1.player.flow_trait_sfds9,introduction.1.player.flow_trait_sfds10,introduction.1.player.rcsq_sleep,headphones_setup_1.1.player.ux_comfort,headphones_setup_1.1.player.ux_speed,headphones_setup_1.1.player.ux_ease,headphones_setup_1.1.player.ux_look,headphones_setup_1.1.player.wx_public,headphones_setup_1.1.player.wx_private,headphones_setup_1.1.player.wx_conversation,headphones_setup_1.1.player.wx_others,headphones_setup_1.1.player.tlx_single,headphones_setup_1.1.player.sus_01,headphones_setup_1.1.player.sus_02,headphones_setup_1.1.player.sus_03,headphones_setup_1.1.player.sus_04,headphones_setup_1.1.player.sus_05,headphones_setup_1.1.player.sus_06,headphones_setup_1.1.player.sus_07,headphones_setup_1.1.player.sus_08,headphones_setup_1.1.player.sus_09,headphones_setup_1.1.player.sus_10,headphones_setup_1.1.player.mr_mood,headphones_setup_1.1.player.mr_sleepy,headphones_setup_1.1.player.mr_motivy,headphones_setup_1.1.player.mf_single,headphones_setup_1.1.player.mf_01,headphones_setup_1.1.player.mf_02,headphones_setup_1.1.player.mf_03,headphones_setup_1.1.player.mf_04,headphones_setup_1.1.player.rest_actions_eo,headphones_setup_1.1.player.rest_actions_ec,headphones_setup_1.1.subsession.round_number,mathTask_1.1.player.fss06,mathTask_1.1.player.fss08,mathTask_1.1.player.fss09,mathTask_1.1.player.tlx_single,mathTask_1.1.player.difficulty,mathTask_1.1.player.mr_mood,mathTask_1.1.player.mr_sleepy,mathTask_1.1.player.mr_motivy,mathTask_1.1.player.mf_single,mathTask_1.1.player.math_actions,mathTask_1.1.subsession.round_number,mathTask_1.2.player.fss06,mathTask_1.2.player.fss08,mathTask_1.2.player.fss09,mathTask_1.2.player.tlx_single,mathTask_1.2.player.difficulty,mathTask_1.2.player.mr_mood,mathTask_1.2.player.mr_sleepy,mathTask_1.2.player.mr_motivy,mathTask_1.2.player.mf_single,mathTask_1.2.player.math_actions,mathTask_1.2.subsession.round_number,mathTask_1.3.player.fss06,mathTask_1.3.player.fss08,mathTask_1.3.player.fss09,mathTask_1.3.player.tlx_single,mathTask_1.3.player.difficulty,mathTask_1.3.player.mr_mood,mathTask_1.3.player.mr_sleepy,mathTask_1.3.player.mr_motivy,mathTask_1.3.player.mf_single,mathTask_1.3.player.math_actions,mathTask_1.3.subsession.round_number,mathTask_1.4.player.fss06,mathTask_1.4.player.fss08,mathTask_1.4.player.fss09,mathTask_1.4.player.tlx_single,mathTask_1.4.player.difficulty,mathTask_1.4.player.mr_mood,mathTask_1.

In [4]:
# Extract client side timestamps from dedicated otree log columns
def get_actionlog_times(logs_df):
    # Get the timestamps from logging fields
    timestamps = logs_df.filter(regex='actions').dropna(axis=1, how='all').reset_index(drop=True)
    # display(timestamps)
    
    def extract_times(s):
        s = s[0].split(';') # Split string
        s = s[1:] # Remove first, empty entry
        # print(s)

        # Convert into DataFrame
        n_messages = int(len(s)/3)
        messages = []
        ts = []
        ts_unix = []
        for i in range(0, n_messages):
            step = i*3
            messages.append(s[step])
            ts.append(s[step+1])
            ts_unix.append(s[step+2])

        df = pd.DataFrame({'Message':messages, 
                           'TS':ts,
                           'TS_UNIX':ts_unix})

        # Convert unix timestamp to correct datetime
        df['TS'] = pd.to_datetime(df['TS_UNIX'], unit='ms', utc=True).dt.tz_convert('Europe/Berlin')
        # print(df)

        return df

    # Parse TS cols
    # TODO: This code could be nicer using a split-apply-combine logic...
    timestamps_parsed = pd.DataFrame()
    for phase in timestamps.columns:
        msgs = extract_times(timestamps[phase])
        msgs['Exp_Phase'] = phase.replace('xTS', '')
        timestamps_parsed = timestamps_parsed.append(msgs)
        # timestamps_parsed = pd.concat([timestamps_parsed, msgs]) # Had to change this as append is deprecated in pandas since 2.0 -> Alternatively: versioning in requirements.txt 
        # print(timestamps_parsed)
        
    # Remove unnecessary events
    timestamps_parsed = timestamps_parsed[timestamps_parsed.Message.str.contains("taskStart|taskEnd")]
    # display(timestamps_parsed)
    # display(timestamps_parsed['Exp_Phase'].value_counts())
    
    # Restructure DF
    timestamps_parsed.drop(['TS_UNIX'], axis=1, inplace=True)
    timestamps_parsed['Message'] = timestamps_parsed['Message'].str.replace('task', '', regex=False) # Remove the substring "task"
    timestamps_parsed = timestamps_parsed.pivot(index='Exp_Phase', columns='Message', values='TS').reset_index() # Pivot the DataFrame
    
    timestamps_parsed[['app_name', 'round_number', 'player', 'page_name']] = timestamps_parsed['Exp_Phase'].str.split('.', expand=True)
    timestamps_parsed.drop(['Exp_Phase', 'player'], axis=1, inplace=True)
    timestamps_parsed['token'] = logs_df['participant.token'][0]
    
    # Add mapping to three EEG recordings
    mapping = {
        "_1": "rec1",
        "_2": "rec2",
        "_3": "rec3"
    }

    timestamps_parsed["rec"] = timestamps_parsed["app_name"].str[-2:].map(mapping)
    timestamps_parsed["app_name"] = timestamps_parsed["app_name"].str.replace(r"_[123]$", "", regex=True)
    # display(timestamps_parsed)
    
    # Structure IDs
    timestamps_parsed['page_name'].replace('rest_actions_', '', regex=True, inplace=True)
    timestamps_parsed['page_name'].replace('_actions', '', regex=True, inplace=True)
    timestamps_parsed['Exp_Phase'] = timestamps_parsed['app_name'] + "_" + timestamps_parsed['page_name'] + "_" + timestamps_parsed['round_number']
    
    return timestamps_parsed

# EEG Settings & Functions

In [5]:
# EEG File metdata ------
chans = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8'] # Varies for the two treatments

fs = 250 # Sampling frequency

# PSD parameters
min_freq, max_freq = 1, 45
psd_normalize = False
psd_window_sec, psd_overlap_pct = 1, 0.75

In [6]:
# Label and cut the EEG data
def label_data(eeg, timestamps, cut=True, report=True):        
    # Annotate the data by checking conditions
    for i, row in timestamps.iterrows():        
        idx = (eeg['TS_UNIX'] >= row['Start']) & (eeg['TS_UNIX'] < row['End'])
        eeg.loc[idx, 'Condition'] = row['Exp_Phase']

    # Drop entries where condition is NA
    if cut: eeg = eeg.dropna(subset=['Condition'])
    
    # Check if annotations are correct
    if report:
        display(eeg['Condition'].value_counts())
    
    return eeg

In [7]:
# Assess ASSR effect across chans
def get_assr_snr(channel_df):
    # Make sure we're working on a copy - to avoid warnings
    channel_df = channel_df.copy()
    
    '''
    # TODO: Should also check Mikkelsen2015 data processing: They exclude some chans without a clear peak at 40Hz
    Mikkelsen2015: Using the ASSR paradigm, we then estimated the signal-to-
    noise ratios (SNR) for both scalp and ear-EEG setups, whereby
    the SNR was defined as the diﬀerence between the logarithm of
    the power at 40 Hz (the signal) and the logarithm of the average
    power in 5 Hz intervals around 40 Hz (the noise floor).
    '''
    # dev = eo_ec_signals[eo_ec_signals.Condition=="headphones_setup_1_eo_1"]["A1"]
    # Split in half
    # TODO: Could also get a more precise indicator of when the ASSR stimulus started - have a timestamp
    first_half, second_half = channel_df.iloc[:channel_df.shape[0]//2], channel_df.iloc[channel_df.shape[0]//2:]   
    # display(first_half)
    # display(second_half)
    
    # Extract SNR
    def calculate_assr_snr(signal):
        # Calculate PSD
        psd = nk.signal_psd(signal, sampling_rate=fs, min_frequency = 1, max_frequency=100, normalize=False, window=1)
        # TODO: Should I take the logarithm after psd extraction like Mikkelsen2015?
        assr_40hz_power = psd.query("40.5 < Frequency < 41.5").Power.mean()
        assr_noise_floor_below = psd.query("36 <= Frequency <= 40.5").Power.mean()
        assr_noise_floor_above = psd.query("41.5 <= Frequency <= 46").Power.mean()
        assr_noise_floor = np.mean((assr_noise_floor_below, assr_noise_floor_above))
        snr = assr_40hz_power - assr_noise_floor
        return snr

    return pd.Series({
        "SNR_First_Half": calculate_assr_snr(first_half.mV),
        "SNR_Second_Half": calculate_assr_snr(second_half.mV)
    })

In [8]:
# Process recordings
def process_recording(active_recording):
    # Filter the otree log
    otree_active = otree_df[otree_df['participant.token']==active_recording.ID].reset_index(drop=True)
    display(otree_active)

    # Extract timings (2 different options)
    timestamps = get_actionlog_times(otree_active)
    # Filter for the active recording
    timestamps = timestamps[timestamps.rec==active_recording.Run]
    
    # Load EEG data
    rec_folder = main_folder + active_recording.Rec_folder
    eeg, load_report_dict = flows.load_exg_streams_data(rec_folder, chanlist=chans, report=True, cleanFile=True)

    # Now annotate (add condition timings) and cut the EEG Data
    eeg = label_data(eeg, timestamps, cut=True, report=False)
    
    # TODO: Process signals now and extract features of interest
    # TODO: Berger effect during EC/EO rest
    # - So far only visualization
    # - Can use flows.get_channel_band_power() to extract band powers instead
    flows.plot_PSD_two_conditions(eeg, chans, 'headphones_setup_ec_1', 'headphones_setup_eo_1', 
                                  fs, min_freq, max_freq, psd_normalize)
    
    # TODO: ASSR during rest
    # - Not yet checked for correctness...
    # Filter for EC/EO conditions
    print(eeg['Condition'])
    eo_ec_signals = eeg[eeg.Condition.str.contains("eo|ec")]
    print( "signals:",  eo_ec_signals)
    # Do for each channel and condition
    eo_ec_signals = eo_ec_signals.drop(['TS_UNIX'], axis=1).melt(id_vars="Condition", value_vars=chans, var_name="Channel", value_name="mV")

    snr_df = (
        eo_ec_signals
        .groupby(["Condition", "Channel"])
        .apply(lambda group: get_assr_snr(group.drop(columns=["Condition", "Channel"])))
        .reset_index()
    )
    snr_df

    # Add the SNR difference to the DataFrame
    snr_df["SNR_Diff"] = snr_df["SNR_Second_Half"] - snr_df["SNR_First_Half"]

    # Plot
    fig = px.box(
        snr_df,
        x="Channel",
        y="SNR_Diff",
        points="all",  # show individual data points
        color="Channel",  # optional: adds color by channel
        title="SNR Change (Second Half - First Half) by Channel",
        labels={"SNR_Diff": "ΔSNR", "Channel": "EEG Channel"}
    )

    fig.show()
    
    # TODO: Other features...

# Process the recordings by looping through the mapping file
# For a single participant
active_recording = sessions.iloc[9]
process_recording(active_recording)

"""
# Loop through all recordings
for index, row in sessions.iterrows():
    process_recording(row)
"""

print('--- All done!\n')

,participant.code,participant._index_in_pages,participant._max_page_index,participant.time_started_utc,participant.token,session.config.name,setup.1.player.headset_color,setup.1.player.token,introduction.1.player.handedness,introduction.1.player.english,introduction.1.player.multilingual,introduction.1.player.age,introduction.1.player.gender,introduction.1.player.glasses,introduction.1.player.eye_sight,introduction.1.player.color_sight,introduction.1.player.hearing,introduction.1.player.occupation,introduction.1.player.education_level,introduction.1.player.headsize,introduction.1.player.weight,introduction.1.player.height,introduction.1.player.hair_style_top,introduction.1.player.hair_style_ears,introduction.1.player.hair_type,introduction.1.player.hair_density,introduction.1.player.beard_style,introduction.1.player.hair_products,introduction.1.player.skin_oily_dry_1,introduction.1.player.skin_oily_dry_2,introduction.1.player.skin_resistant_sensitive_1,introduction.1.player.skin_resistant_sensitive_2,introduction.1.player.skin_products,introduction.1.player.time_since_meal,introduction.1.player.time_since_hydration,introduction.1.player.time_since_caffeine,introduction.1.player.time_since_nicotine,introduction.1.player.time_since_alcohol,introduction.1.player.flow_trait_single,introduction.1.player.flow_trait_sfds1,introduction.1.player.flow_trait_sfds2,introduction.1.player.flow_trait_sfds3,introduction.1.player.flow_trait_sfds4,introduction.1.player.flow_trait_sfds5,introduction.1.player.flow_trait_sfds6,introduction.1.player.flow_trait_sfds7,introduction.1.player.flow_trait_sfds8,introduction.1.player.flow_trait_sfds9,introduction.1.player.flow_trait_sfds10,introduction.1.player.rcsq_sleep,headphones_setup_1.1.player.ux_comfort,headphones_setup_1.1.player.ux_speed,headphones_setup_1.1.player.ux_ease,headphones_setup_1.1.player.ux_look,headphones_setup_1.1.player.wx_public,headphones_setup_1.1.player.wx_private,headphones_setup_1.1.player.wx_conversation,headphones_setup_1.1.player.wx_others,headphones_setup_1.1.player.tlx_single,headphones_setup_1.1.player.sus_01,headphones_setup_1.1.player.sus_02,headphones_setup_1.1.player.sus_03,headphones_setup_1.1.player.sus_04,headphones_setup_1.1.player.sus_05,headphones_setup_1.1.player.sus_06,headphones_setup_1.1.player.sus_07,headphones_setup_1.1.player.sus_08,headphones_setup_1.1.player.sus_09,headphones_setup_1.1.player.sus_10,headphones_setup_1.1.player.mr_mood,headphones_setup_1.1.player.mr_sleepy,headphones_setup_1.1.player.mr_motivy,headphones_setup_1.1.player.mf_single,headphones_setup_1.1.player.mf_01,headphones_setup_1.1.player.mf_02,headphones_setup_1.1.player.mf_03,headphones_setup_1.1.player.mf_04,headphones_setup_1.1.player.rest_actions_eo,headphones_setup_1.1.player.rest_actions_ec,headphones_setup_1.1.subsession.round_number,mathTask_1.1.player.fss06,mathTask_1.1.player.fss08,mathTask_1.1.player.fss09,mathTask_1.1.player.tlx_single,mathTask_1.1.player.difficulty,mathTask_1.1.player.mr_mood,mathTask_1.1.player.mr_sleepy,mathTask_1.1.player.mr_motivy,mathTask_1.1.player.mf_single,mathTask_1.1.player.math_actions,mathTask_1.1.subsession.round_number,mathTask_1.2.player.fss06,mathTask_1.2.player.fss08,mathTask_1.2.player.fss09,mathTask_1.2.player.tlx_single,mathTask_1.2.player.difficulty,mathTask_1.2.player.mr_mood,mathTask_1.2.player.mr_sleepy,mathTask_1.2.player.mr_motivy,mathTask_1.2.player.mf_single,mathTask_1.2.player.math_actions,mathTask_1.2.subsession.round_number,mathTask_1.3.player.fss06,mathTask_1.3.player.fss08,mathTask_1.3.player.fss09,mathTask_1.3.player.tlx_single,mathTask_1.3.player.difficulty,mathTask_1.3.player.mr_mood,mathTask_1.3.player.mr_sleepy,mathTask_1.3.player.mr_motivy,mathTask_1.3.player.mf_single,mathTask_1.3.player.math_actions,mathTask_1.3.subsession.round_number,mathTask_1.4.player.fss06,mathTask_1.4.player.fss08,mathTask_1.4.player.fss09,mathTask_1.4.player.tlx_single,mathTask_1.4.player.difficulty,mathTask_1.4.player.mr_mood,mathTask_1.

/var/folders/ml/fyq5x5t55wx4129dn03n32hm0000gn/T/ipykernel_53174/3174027199.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timestamps_parsed = timestamps_parsed.append(msgs)
/var/folders/ml/fyq5x5t55wx4129dn03n32hm0000gn/T/ipykernel_53174/3174027199.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timestamps_parsed = timestamps_parsed.append(msgs)
/var/folders/ml/fyq5x5t55wx4129dn03n32hm0000gn/T/ipykernel_53174/3174027199.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timestamps_parsed = timestamps_parsed.append(msgs)
/var/folders/ml/fyq5x5t55wx4129dn03n32hm0000gn/T/ipykernel_53174/3174027199.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

n times samples loss: 589
Total samples lost: 171.0
Recording duration (hh:mm:ss.ms): 0:36:34.832999
Estimated fs: 250.43
n times gaps with >1sec: 0


/Users/Sam/miniconda3/envs/charite/lib/python3.12/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


32975     headphones_setup_eo_1
32976     headphones_setup_eo_1
32977     headphones_setup_eo_1
32978     headphones_setup_eo_1
32979     headphones_setup_eo_1
                  ...          
513698           clickTask_ec_8
513699           clickTask_ec_8
513700           clickTask_ec_8
513701           clickTask_ec_8
513702           clickTask_ec_8
Name: Condition, Length: 255418, dtype: object
signals:                    A1            A2            A3            A4            A5  \
32975  -143157.201476 -93055.095627   2794.057469 -78943.768772 -30611.876383   
32976  -143135.967319 -93050.513519   2795.063298 -78948.082658 -30614.312724   
32977  -143149.892455 -93051.251127   2772.599795 -78943.277033 -30611.496404   
32978  -143162.163563 -93054.693295   2770.565786 -78946.383926 -30619.252459   
32979  -143151.546485 -93048.010124   2766.050734 -78931.877644 -30609.350636   
...               ...           ...           ...           ...           ...   
513698  -35627.943125 -57

--- All done!

